In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
pd.read_csv('./data/train.csv')

# Part B > Clustering

In [ ]:
df2 = pd.read_csv('./data/Mall_Customers.csv', index_col=0)
df2['Male'] = df2['Genre'] == 'Male'
df2.head()

In [ ]:
import seaborn as sns
from sklearn.metrics import silhouette_score
cols = df2.columns.drop(['Genre', 'Male'])
n = len(cols)
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(10, 8))
c = 0
for i in range(n):
    for j in range(n):
        if i != j:
            sns.scatterplot(data=df2, x=cols[i], y=cols[j], hue=df2['Male'], ax=ax[c % 2, c % 3])
            c += 1

In [ ]:
cluster_params = [4, 5]
fig, ax = plt.subplots(ncols=2, figsize=(10, 8))
for i, cl in enumerate(cluster_params):
    model = KMeans(n_clusters=cl).fit(X=df2[['Annual Income (k$)', 'Spending Score (1-100)']])
    y_hat = model.predict(df2[['Annual Income (k$)', 'Spending Score (1-100)']])
    sns.scatterplot(data=df2, x='Annual Income (k$)', y='Spending Score (1-100)', hue=y_hat, ax=ax[i])
    print(f'Silhouette Score ({cl}):', silhouette_score(X=df2[['Annual Income (k$)', 'Spending Score (1-100)']], labels=model.labels_, metric='euclidean'))